Load Real Estate Data

In [49]:
import pandas as pd

df = pd.read_csv("/content/600K US Housing Properties.csv")

df.head

<ipython-input-49-8083e1eafc05>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/600K US Housing Properties.csv")


<bound method NDFrame.head of                                              property_url  property_id  \
0       https://www.zillow.com/homedetails/3-Plat-83-1...   2064212272   
1       https://www.zillow.com/homedetails/117-3rd-St-...    249518113   
2       https://www.zillow.com/homedetails/LOT-2A-Fron...   2077729913   
3       https://www.zillow.com/homedetails/LOT-A-Plat-...   2067488502   
4       https://www.zillow.com/homedetails/335-Cassiar...    249518139   
...                                                   ...          ...   
599995  https://www.zillow.com/homedetails/6317-Virgin...      4303608   
599996  https://www.zillow.com/homedetails/LOT-16-Ray-...   2078144422   
599997  https://www.zillow.com/homedetails/12S135-Lemo...      4604875   
599998  https://www.zillow.com/homedetails/4701-Abbey-...     65578365   
599999  https://www.zillow.com/homedetails/321-Allen-A...      4418670   

                                       address         street_name apartment  \
0

In [50]:
# Assuming you have already loaded or created your DataFrame 'df'

# Convert acres to square feet
df['land_space'] = df.apply(lambda row: row['land_space'] * 43560 if row['land_space_unit'] == 'acres' else row['land_space'], axis=1)

# Drop the 'land_space_unit' column
cols_dropped = ["land_space_unit"]
df = df.drop(columns=cols_dropped)

relevant_columns = ['address', 'bedroom_number', 'bathroom_number', 'price_per_unit', 'living_space', 'price', 'postcode', 'latitude', 'longitude']

# Drop columns not in the 'relevant_columns' list
df = df[relevant_columns]

print(df.shape)

df.head()

(600000, 9)


,address,bedroom_number,bathroom_number,price_per_unit,living_space,price,postcode,latitude,longitude
0,"3 Plat #83-10, Wrangell, AK 99929",NaN,NaN,NaN,NaN,135000.0,99929,NaN,NaN
1,"117 3rd St, Wrangell, AK 99929",3.0,3.0,237.0,2478.0,589500.0,99929,56.474518,-132.38600
2,"LOT 2A Front St, Wrangell, AK 99929",NaN,0.0,NaN,NaN,99999.0,99929,56.471638,-132.38686
3,"LOT A Plat #2009-6, Wrangell, AK 99929",3.0,1.0,330.0,1500.0,495000.0,99929,NaN,NaN
4,"335 Cassiar St, Wrangell, AK 99929",5.0,3.0,194.0,2080.0,405000.0,99929,56.475697,-132.38905


In [51]:
df = df.dropna(subset=relevant_columns)

# df.dtypes

In [52]:
# Function to convert postcode to int or None if it can't be converted
def try_cast_to_int(postcode):
    try:
        return int(postcode)
    except ValueError:
        return None

# Apply the function to the 'postcode' column , 'living_space', 'price', 'postcode''
df['postcode'] = df['postcode'].apply(try_cast_to_int)
df['bedroom_number'] = df['bedroom_number'].apply(try_cast_to_int)
df['bathroom_number'] = df['bathroom_number'].apply(try_cast_to_int)
df['price_per_unit'] = df['price_per_unit'].apply(try_cast_to_int)
df['price'] = df['price'].apply(try_cast_to_int)
df['living_space'] = df['living_space'].apply(try_cast_to_int)

# Drop rows where postcode couldn't be converted to int
df = df.dropna(subset=['postcode'])
df = df.dropna(subset=['bedroom_number'])
df = df.dropna(subset=['bathroom_number'])
df = df.dropna(subset=['price_per_unit'])
df = df.dropna(subset=['price'])
df = df.dropna(subset=['living_space'])

# Print the resulting DataFrame
print(df)

<ipython-input-52-deaed196e5bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['postcode'] = df['postcode'].apply(try_cast_to_int)


                                       address  bedroom_number  \
1               117 3rd St, Wrangell, AK 99929               3   
4           335 Cassiar St, Wrangell, AK 99929               5   
7           532.5 Front St, Wrangell, AK 99929               3   
44        501 Noseeum St, Petersburg, AK 99833               1   
48           905 Odin St, Petersburg, AK 99833               3   
...                                        ...             ...   
599993  611 Hickory Ln, West Chicago, IL 60185               4   
599995    6317 Virginia Ln, Matteson, IL 60443               4   
599997      12S135 Lemont Rd, Lemont, IL 60439               3   
599998       4701 Abbey Ln, Matteson, IL 60443               5   
599999   321 Allen Ave, West Chicago, IL 60185               4   

        bathroom_number  price_per_unit  living_space   price  postcode  \
1                     3             237          2478  589500   99929.0   
4                     3             194          2080  40

In [53]:
# df.dtypes

# Count occurrences of each postcode
postcode_counts = df['postcode'].value_counts()

# Print the counts
print(postcode_counts)

# Assuming you have a DataFrame called df

# Determine the size of the reduced dataset (1/100 of the original size)
reduced_size = len(df) // 100

# Randomly sample the DataFrame to create a reduced dataset
df = df.sample(n=reduced_size, random_state=42)

# Now, reduced_df contains 1/10th of the original data in a random sample

75126.0    872
73099.0    816
78130.0    801
60611.0    773
78253.0    746
          ... 
70759.0      1
65461.0      1
62538.0      1
62963.0      1
78886.0      1
Name: postcode, Length: 9858, dtype: int64


In [54]:
from sklearn.model_selection import train_test_split

x = df[['bedroom_number', 'bathroom_number', 'living_space', 'postcode']]

y = df[['price']].values.ravel()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [55]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error


# # Classification with Logistic Regression
# cls_model = LogisticRegression()
# cls_model.fit(x_train, y_train)
# # y_pred_cls = cls_model.predict(x_test)
# # accuracy_cls = accuracy_score(y_test, y_pred_cls)
# print(f'Accuracy: {cls_model.score(x_test, y_test):.2f}')



In [56]:
# Regression with Linear Regression
reg_model = LinearRegression()
reg_model.fit(x_train, y_train)

# y_pred_reg = reg_model.predict(x_test)
# mse_reg = mean_squared_error(y_test, y_pred_reg)
print(f'Accuracy: {reg_model.score(x_test, y_test):.2f}')



Accuracy: 0.25


In [57]:
df_with_predictions = df.copy()

df_with_predictions['bedroom_number'] += 1
df_with_predictions['bathroom_number'] += 1
df_with_predictions['living_space'] += 250

# Predict the prices for the modified properties
predicted_prices = reg_model.predict(df_with_predictions[['bedroom_number', 'bathroom_number', 'living_space', 'postcode']])

# Append the predictions as a new column
df_with_predictions['predicted_price'] = predicted_prices

df_with_predictions['predicted_price'] = df_with_predictions['predicted_price'].apply(try_cast_to_int)
df_with_predictions['predicted_price'] = df_with_predictions['predicted_price'].abs()

# Display the updated DataFrame with predictions
print(df_with_predictions)

                                                 address  bedroom_number  \
352348         2004 N Mission Cir, Friendswood, TX 77546               5   
413006           355 County Road 3900, Hawkins, TX 75765               4   
53331   45 Eagle Crest Dr APT 405, Lake Oswego, OR 97035               3   
359568    9602 Oakland Lake Way, Missouri City, TX 77459               6   
284557         12664 W Bowles Place, Littleton, CO 80127               3   
...                                                  ...             ...   
54807       1850 Ala Moana Blvd #923, Honolulu, HI 96815               1   
158990               15619 Risley St, Whittier, CA 90603               4   
376081         1211 Caroline St #1103, Houston, TX 77002               3   
16241                  3516 S L Street, Tacoma, WA 98418               5   
582179         2819 W Rosemont Ave #G, Chicago, IL 60659               3   

        bathroom_number  price_per_unit  living_space    price  postcode  \
352348     

In [58]:
# Save the DataFrame to a .csv file
df_with_predictions.to_csv('predicted_prices.csv', index=True)

In [59]:
# # Classification with Support Vector Machine
# cls_model = SVC()
# cls_model.fit(x_train, y_train)
# y_pred_cls = cls_model.predict(x_test)
# accuracy_cls = accuracy_score(y_test, y_pred_cls)
# print(f'Classification Accuracy: {accuracy_cls:.2f}')

In [60]:
# # Regression with Gradient Boosting
# reg_model = GradientBoostingRegressor()
# reg_model.fit(x_train, y_train)
# y_pred_reg = reg_model.predict(x_test)
# mse_reg = mean_squared_error(y_test, y_pred_reg)
# print(f'Regression Mean Squared Error: {mse_reg:.2f}')